# European Engine

In [1]:
from QuantLib import *
import datetime
import re
import utils 
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from yahoo_fin.stock_info import get_quote_table
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
sys.path.insert(0, '../scripts/')
import utils as ut
from database import Stock 
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
from pandas.tseries.offsets import BDay
end = pd.datetime.today().date()
start = end - 252 * BDay() * 1 # One year historical data

In [67]:
np.std(np.log(Stock("AAPL", start, end).df.Close.pct_change() + 1)) * 252 ** 0.5

0.3596121989793802

Building the option requires only the specification of its contract, so its payoff (it's a call option with strike at 100) and its exercise, three months from today's date. Market data will be selected and passed later, depending on the calculation methods.

In [87]:
def european_engine(ticker, option="Call"):
                
    info = get_quote_table(ticker)
    current_price = info["Quote Price"]
    yield_re = re.compile(r"\((?P<value>(\d+\.\d+))%\)")
    try:
        dividend_rate = float(yield_re.search(info["Forward Dividend & Yield"])["value"])
    except (KeyError, ValueError, TypeError):
        dividend_rate = 0.0      
    
    today = Date.todaysDate()
    Settings.instance().evaluationDate = today
    
    stk_sigma = np.std(np.log(Stock("AAPL", start, end).df.Close.pct_change() + 1)) * 252 ** 0.5

    def create_call(row):
        u = SimpleQuote(current_price)
        r = SimpleQuote(0.01)
        sigma = SimpleQuote(stk_sigma)
        option = EuropeanOption(PlainVanillaPayoff(Option.Call, row["strike"]), EuropeanExercise(Date(expiration.day, expiration.month, expiration.year)))
        riskFreeCurve = FlatForward(0, TARGET(), QuoteHandle(r), Actual360())
        volatility = BlackConstantVol(0, TARGET(), QuoteHandle(sigma), Actual360())
        process = BlackScholesProcess(QuoteHandle(u),YieldTermStructureHandle(riskFreeCurve),BlackVolTermStructureHandle(volatility))
        process.dividendYield().zeroRate(dividend_rate, Continuous)
        
        engine = AnalyticEuropeanEngine(process)
        option.setPricingEngine(engine)
        
        npv = option.NPV()
        delta = option.delta()
        gamma = option.gamma()
        theta = option.theta()
        vega = option.vega()
        rho = option.rho()
        
        return {"Underlying": current_price,
                "Ticker": ticker,
                "Type": row["type"],
                "Expiration": row['expirationDate'],
                "YTE": row["yte"],
                "DTE": row["dte"],
                "Strike": row["strike"],
                "Last": row["lastPrice"],
                "Bid": row["bid"],
                "Ask": row["ask"],
                "Midpoint": (row['bid'] + row['ask']) / 2,
                "Spread": row['ask'] - row['bid'],
                "IV": row["impliedVolatility"],  
                "NPV": npv,
                "Delta": delta,
                "Gamma": gamma,
                "Theta":  theta / 365,
                "Vega": vega,
                "Rho": rho,
                "AKA": row["contractSymbol"], 
                "Intrinsic_Value":  current_price - row["strike"],
                "Time_Value": row["lastPrice"] - (current_price- row["strike"])}
        
    def create_put(row):
        u = SimpleQuote(current_price)
        r = SimpleQuote(0.0001)
        sigma = SimpleQuote(stk_sigma)
        option = EuropeanOption(PlainVanillaPayoff(Option.Put, row["strike"]), EuropeanExercise(Date(expiration.day, expiration.month, expiration.year)))
        riskFreeCurve = FlatForward(0, TARGET(), QuoteHandle(r), Actual360())
        volatility = BlackConstantVol(0, TARGET(), QuoteHandle(sigma), Actual360())
        process = BlackScholesProcess(QuoteHandle(u),YieldTermStructureHandle(riskFreeCurve),BlackVolTermStructureHandle(volatility))
        process.dividendYield().zeroRate(dividend_rate, Continuous)
        engine = AnalyticEuropeanEngine(process)
        option.setPricingEngine(engine)
        npv = option.NPV()
        delta = option.delta()
        gamma = option.gamma()
        theta = option.theta()
        vega = option.vega()
        rho = option.rho()
        
        return {"Underlying": current_price,
                "Ticker": ticker,
                "Type": row["type"],
                "Expiration": row['expirationDate'],
                "YTE": row["yte"],
                "DTE": row["dte"],
                "Strike": row["strike"],
                "Last": row["lastPrice"],
                "Bid": row["bid"],
                "Ask": row["ask"],
                "Midpoint": (row['bid'] + row['ask']) / 2,
                "Spread": row['ask'] - row['bid'],
                "IV": row["impliedVolatility"],  
                "NPV": npv,
                "Delta": delta,
                "Gamma": gamma,
                "Theta":  theta / 365,
                "Vega": vega,
                "Rho": rho,
                "AKA": row["contractSymbol"], 
                "Intrinsic_Value":  row["strike"] - current_price, 
                "Time_Value": row["lastPrice"] - (row["strike"] - current_price)}          
        
    options_= pd.DataFrame()       
    
    if option == "Call":
        
        tk = yf.Ticker(ticker)
        exps = tk.options
        
        for e in exps:
            opt = tk.option_chain(e)
            calls = pd.DataFrame().append(opt.calls)
            calls['expirationDate'] = e
            expiration =  pd.to_datetime(e) + datetime.timedelta(days = 1)
            calls['expirationDate'] = expiration
            calls['yte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days / 365
            calls['dte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days
            calls[['bid', 'ask', 'strike']] = calls[['bid', 'ask', 'strike']].apply(pd.to_numeric)
            calls['type'] = "Call"
            calls = calls.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
            options = calls.apply(create_call, axis=1, result_type="expand")
            options_ = options_.append(options, ignore_index=True) 
    else:    
        
        tk = yf.Ticker(ticker)
        exps = tk.options
        
        for e in exps:
            opt = tk.option_chain(e)
            puts = pd.DataFrame().append(opt.puts)
            puts['expirationDate'] = e
            expiration =  pd.to_datetime(e) + datetime.timedelta(days = 1)
            puts['expirationDate'] = expiration
            puts['yte'] = (puts['expirationDate'] - datetime.datetime.today()).dt.days / 365
            puts['dte'] = (puts['expirationDate'] - datetime.datetime.today()).dt.days
            puts[['bid', 'ask', 'strike']] = puts[['bid', 'ask', 'strike']].apply(pd.to_numeric)
            puts['type'] = "Put"
            puts = puts.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
            options = puts.apply(create_put, axis=1, result_type="expand")
            options_ = options_.append(options, ignore_index=True) 
    return  options_

In [88]:
call = european_engine("AAPL", option="Call")

In [89]:
call

,Underlying,Ticker,Type,Expiration,YTE,DTE,Strike,Last,Bid,Ask,...,IV,NPV,Delta,Gamma,Theta,Vega,Rho,AKA,Intrinsic_Value,Time_Value
0,121.2100,AAPL,Call,2021-04-02,0.0137,5,60.0000,60.9000,59.1500,62.7000,...,3.8887,61.2167,1.0000,0.0000,-0.0016,0.0000,0.6666,AAPL210401C00060000,61.2100,-0.3100
1,121.2100,AAPL,Call,2021-04-02,0.0137,5,65.0000,57.9000,54.0000,57.7000,...,3.5107,56.2172,1.0000,0.0000,-0.0018,0.0000,0.7221,AAPL210401C00065000,56.2100,1.6900
2,121.2100,AAPL,Call,2021-04-02,0.0137,5,70.0000,51.2500,49.0000,52.7000,...,3.1611,51.2178,1.0000,0.0000,-0.0019,0.0000,0.7777,AAPL210401C00070000,51.2100,0.0400
3,121.2100,AAPL,Call,2021-04-02,0.0137,5,80.0000,41.0500,39.3500,42.7000,...,2.5254,41.2189,1.0000,0.0000,-0.0022,0.0000,0.8888,AAPL210401C00080000,41.2100,-0.1600
4,121.2100,AAPL,Call,2021-04-02,0.0137,5,85.0000,37.8000,34.0000,37.7000,...,2.2324,36.2194,1.0000,0.0000,-0.0023,0.0000,0.9443,AAPL210401C00085000,36.2100,1.5900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,121.2100,AAPL,Call,2023-03-18,1.9726,720,185.0000,6.5000,6.5500,7.1500,...,0.3379,9.0021,0.2951,0.0056,-0.0153,59.1116,53.4561,AAPL230317C00185000,-63.7900,70.2900
1121,121.2100,AAPL,Call,2023-03-18,1.9726,720,190.0000,5.7000,5.9500,7.8500,...,0.3622,8.3080,0.2772,0.0054,-0.0148,57.3853,50.5222,AAPL230317C00190000,-68.7900,74.4900
1122,121.2100,AAPL,Call,2023-03-18,1.9726,720,195.0000,5.5500,4.0000,6.0000,...,0.3386,7.6692,0.2604,0.0053,-0.0144,55.6050,47.7167,AAPL230317C00195000,-73.7900,79.3400
1123,121.2100,AAPL,Call,2023-03-18,1.9726,720,200.0000,5.2000,5.1000,7.5000,...,0.3777,7.0814,0.2445,0.0051,-0.0139,53.7875,45.0395,AAPL230317C00200000,-78.7900,83.9900


In [90]:
put = european_engine("AAPL", option="Put")

In [91]:
put

,Underlying,Ticker,Type,Expiration,YTE,DTE,Strike,Last,Bid,Ask,...,IV,NPV,Delta,Gamma,Theta,Vega,Rho,AKA,Intrinsic_Value,Time_Value
0,121.2100,AAPL,Put,2021-04-02,0.0137,5,60.0000,0.0100,0.0000,1.7600,...,3.4482,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,AAPL210401P00060000,-61.2100,61.2200
1,121.2100,AAPL,Put,2021-04-02,0.0137,5,65.0000,0.0100,0.0000,0.0100,...,1.5625,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,AAPL210401P00065000,-56.2100,56.2200
2,121.2100,AAPL,Put,2021-04-02,0.0137,5,70.0000,0.0100,0.0000,0.0100,...,1.3750,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,AAPL210401P00070000,-51.2100,51.2200
3,121.2100,AAPL,Put,2021-04-02,0.0137,5,75.0000,0.0100,0.0000,0.0100,...,1.2500,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,AAPL210401P00075000,-46.2100,46.2200
4,121.2100,AAPL,Put,2021-04-02,0.0137,5,80.0000,0.0100,0.0000,0.0200,...,1.1563,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,AAPL210401P00080000,-41.2100,41.2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,121.2100,AAPL,Put,2023-03-18,1.9726,720,185.0000,66.8000,71.6500,72.3500,...,0.3636,72.2368,-0.7182,0.0055,-0.0142,57.8421,-318.1363,AAPL230317P00185000,63.7900,3.0100
1089,121.2100,AAPL,Put,2023-03-18,1.9726,720,190.0000,68.8100,75.8500,76.5000,...,0.3603,76.5705,-0.7356,0.0053,-0.0138,56.0383,-331.0149,AAPL230317P00190000,68.7900,0.0200
1090,121.2100,AAPL,Put,2023-03-18,1.9726,720,195.0000,79.5600,76.0000,80.5000,...,0.3530,80.9583,-0.7521,0.0051,-0.0133,54.1919,-343.7599,AAPL230317P00195000,73.7900,5.7700
1091,121.2100,AAPL,Put,2023-03-18,1.9726,720,200.0000,83.7000,80.5000,85.0000,...,0.3539,85.3957,-0.7676,0.0050,-0.0129,52.3191,-356.3721,AAPL230317P00200000,78.7900,4.9100
